# Project 6: Analyzing Stock Sentiment from Twits
## Instructions
Each problem consists of a function to implement and instructions on how to implement the function.  The parts of the function that need to be implemented are marked with a `# TODO` comment.

## Packages
When you implement the functions, you'll only need to you use the packages you've used in the classroom, like [Pandas](https://pandas.pydata.org/) and [Numpy](http://www.numpy.org/). These packages will be imported for you. We recommend you don't add any import statements, otherwise the grader might not be able to run your code.

### Load Packages

In [1]:
import json
import nltk
import os
import random
import re
import torch

import numpy as np

from torch import nn, optim
import torch.nn.functional as F

## Introduction
When deciding the value of a company, it's important to follow the news. For example, a product recall or natural disaster in a company's product chain. You want to be able to turn this information into a signal. Currently, the best tool for the job is a Neural Network. 

For this project, you'll use posts from the social media site [StockTwits](https://en.wikipedia.org/wiki/StockTwits). The community on StockTwits is full of investors, traders, and entrepreneurs. Each message posted is called a Twit. This is similar to Twitter's version of a post, called a Tweet. You'll build a model around these twits that generate a sentiment score.

We've collected a bunch of twits, then hand labeled the sentiment of each. To capture the degree of sentiment, we'll use a five-point scale: very negative, negative, neutral, positive, very positive. Each twit is labeled -2 to 2 in steps of 1, from very negative to very positive respectively. You'll build a sentiment analysis model that will learn to assign sentiment to twits on its own, using this labeled data.

The first thing we should to do, is load the data.

## Import Twits 
### Load Twits Data 
This JSON file contains a list of objects for each twit in the `'data'` field:

```
{'data':
  {'message_body': 'Neutral twit body text here',
   'sentiment': 0},
  {'message_body': 'Happy twit body text here',
   'sentiment': 1},
   ...
}
```

The fields represent the following:

* `'message_body'`: The text of the twit.
* `'sentiment'`: Sentiment score for the twit, ranges from -2 to 2 in steps of 1, with 0 being neutral.


To see what the data look like by printing the first 10 twits from the list. 

In [2]:
with open(os.path.join('..', '..', 'data', 'project_6_stocktwits', 'twits.json'), 'r') as f:
    twits = json.load(f)

print(twits['data'][:10])

[{'message_body': '$FITB great buy at 26.00...ill wait', 'sentiment': 2, 'timestamp': '2018-07-01T00:00:09Z'}, {'message_body': '@StockTwits $MSFT', 'sentiment': 1, 'timestamp': '2018-07-01T00:00:42Z'}, {'message_body': '#STAAnalystAlert for $TDG : Jefferies Maintains with a rating of Hold setting target price at USD 350.00. Our own verdict is Buy  http://www.stocktargetadvisor.com/toprating', 'sentiment': 2, 'timestamp': '2018-07-01T00:01:24Z'}, {'message_body': '$AMD I heard there’s a guy who knows someone who thinks somebody knows something - on StockTwits.', 'sentiment': 1, 'timestamp': '2018-07-01T00:01:47Z'}, {'message_body': '$AMD reveal yourself!', 'sentiment': 0, 'timestamp': '2018-07-01T00:02:13Z'}, {'message_body': '$AAPL Why the drop? I warren Buffet taking out his position?', 'sentiment': 1, 'timestamp': '2018-07-01T00:03:10Z'}, {'message_body': '$BA bears have 1 reason on 06-29 to pay more attention https://dividendbot.com?s=BA', 'sentiment': -2, 'timestamp': '2018-07-01T

### Length of Data
Now let's look at the number of twits in dataset. Print the number of twits below.

In [3]:
"""print out the number of twits"""

# TODO Implement 
print(len(twits['data']))


1548010


### Split Message Body and Sentiment Score

In [4]:
messages = [twit['message_body'] for twit in twits['data']]
# Since the sentiment scores are discrete, we'll scale the sentiments to 0 to 4 for use in our network
sentiments = [twit['sentiment'] + 2 for twit in twits['data']]

## Preprocessing the Data
With our data in hand we need to preprocess our text. These twits are collected by filtering on ticker symbols where these are denoted with a leader $ symbol in the twit itself. For example,

`{'message_body': 'RT @google Our annual look at the year in Google blogging (and beyond) http://t.co/sptHOAh8 $GOOG',
 'sentiment': 0}`

The ticker symbols don't provide information on the sentiment, and they are in every twit, so we should remove them. This twit also has the `@google` username, again not providing sentiment information, so we should also remove it. We also see a URL `http://t.co/sptHOAh8`. Let's remove these too.

The easiest way to remove specific words or phrases is with regex using the `re` module. You can sub out specific patterns with a space:

```python
re.sub(pattern, ' ', text)
```
This will substitute a space with anywhere the pattern matches in the text. Later when we tokenize the text, we'll split appropriately on those spaces.

### Pre-Processing

In [7]:
nltk.download('wordnet')


def preprocess(message):
    """
    This function takes a string as input, then performs these operations: 
        - lowercase
        - remove URLs
        - remove ticker symbols 
        - removes punctuation
        - tokenize by splitting the string on whitespace 
        - removes any single character tokens
    
    Parameters
    ----------
        message : The text message to be preprocessed.
        
    Returns
    -------
        tokens: The preprocessed text into tokens.
    """ 
    #TODO: Implement 
    
    # Lowercase the twit message
    text = message.lower()
    
    # Replace URLs with a space in the message
    # Since URL only resides in the end of sentence we can just find them starting from http/https
    text = re.sub(r"(http|https)+.+"," ",text)
    
    # Replace ticker symbols with a space. The ticker symbols are any stock symbol that starts with $.
    text = re.sub(r"\$[a-zA-Z]{1,5}"," ",text) 
    
    # Replace StockTwits usernames with a space. The usernames are any word that starts with @.
    text = re.sub(r"\@[a-zA-Z]{1,20}"," ",text)

    # Replace everything not a letter with a space
    # \W may include numbers
    text = re.sub(r"[^a-zA-Z]"," ",text)
    
    # Tokenize by splitting the string on whitespace into a list of words
    tokens = text.split()

    # Lemmatize words using the WordNetLemmatizer. You can ignore any word that is not longer than one character.
    wnl = nltk.stem.WordNetLemmatizer()
    tokens = [wnl.lemmatize(t) for t in tokens if len(t)>1]

    
    assert type(tokens) == list, 'Tokens should be list'
    return tokens

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


>Note: You must ensure that after preprocessing the text should NOT include:
- Numbers
- URLs
- Single character tokens
- Ticker symbols (these should be removed even if they don't appear at the beginning)

### Preprocess All the Twits 
Now we can preprocess each of the twits in our dataset. Apply the function `preprocess` to all the twit messages.

In [8]:
# TODO Implement

tokenized = [preprocess(m) for m in messages]
tokenized

[['great', 'buy', 'at', 'ill', 'wait'],
 [],
 ['staanalystalert',
  'for',
  'jefferies',
  'maintains',
  'with',
  'rating',
  'of',
  'hold',
  'setting',
  'target',
  'price',
  'at',
  'usd',
  'our',
  'own',
  'verdict',
  'is',
  'buy'],
 ['heard',
  'there',
  'guy',
  'who',
  'know',
  'someone',
  'who',
  'think',
  'somebody',
  'know',
  'something',
  'on',
  'stocktwits'],
 ['reveal', 'yourself'],
 ['why',
  'the',
  'drop',
  'warren',
  'buffet',
  'taking',
  'out',
  'his',
  'position'],
 ['bear', 'have', 'reason', 'on', 'to', 'pay', 'more', 'attention'],
 ['ok',
  'good',
  'we',
  're',
  'not',
  'dropping',
  'in',
  'price',
  'over',
  'the',
  'weekend',
  'lol'],
 ['daily', 'chart', 'we', 'need', 'to', 'get', 'back', 'to', 'above'],
 ['drop',
  'per',
  'week',
  'after',
  'spike',
  'if',
  'no',
  'news',
  'in',
  'month',
  'back',
  'to',
  'if',
  'bo',
  'then',
  'bingo',
  'what',
  'is',
  'the',
  'odds'],
 ['strong', 'buy'],
 ['short', 'ratio

### Bag of Words
Now with all of our messages tokenized, we want to create a vocabulary and count up how often each word appears in our entire corpus. Use the [`Counter`](https://docs.python.org/3.1/library/collections.html#collections.Counter) function to count up all the tokens.

In [9]:
tokenized_flatten = [w for ts in tokenized for w in ts]
tokenized_flatten

['great',
 'buy',
 'at',
 'ill',
 'wait',
 'staanalystalert',
 'for',
 'jefferies',
 'maintains',
 'with',
 'rating',
 'of',
 'hold',
 'setting',
 'target',
 'price',
 'at',
 'usd',
 'our',
 'own',
 'verdict',
 'is',
 'buy',
 'heard',
 'there',
 'guy',
 'who',
 'know',
 'someone',
 'who',
 'think',
 'somebody',
 'know',
 'something',
 'on',
 'stocktwits',
 'reveal',
 'yourself',
 'why',
 'the',
 'drop',
 'warren',
 'buffet',
 'taking',
 'out',
 'his',
 'position',
 'bear',
 'have',
 'reason',
 'on',
 'to',
 'pay',
 'more',
 'attention',
 'ok',
 'good',
 'we',
 're',
 'not',
 'dropping',
 'in',
 'price',
 'over',
 'the',
 'weekend',
 'lol',
 'daily',
 'chart',
 'we',
 'need',
 'to',
 'get',
 'back',
 'to',
 'above',
 'drop',
 'per',
 'week',
 'after',
 'spike',
 'if',
 'no',
 'news',
 'in',
 'month',
 'back',
 'to',
 'if',
 'bo',
 'then',
 'bingo',
 'what',
 'is',
 'the',
 'odds',
 'strong',
 'buy',
 'short',
 'ratio',
 'is',
 'at',
 'and',
 'short',
 'to',
 'float',
 'is',
 'price',
 '

In [10]:
from collections import Counter


"""
Create a vocabulary by using Bag of words
"""

# TODO: Implement 

bow = Counter(tokenized_flatten)
# Counter with sorted https://iq.opengenus.org/ways-to-sort-counter-in-python/
# vocab = sorted(bow,key=bow.get,reverse=True)# key to use for comparison in sort
# vocab_to_int = {word:i for i,word in enumerate(vocab,1)} #enumerate start with 1
# vocab_to_int

### Frequency of Words Appearing in Message
With our vocabulary, now we'll remove some of the most common words such as 'the', 'and', 'it', etc. These words don't contribute to identifying sentiment and are really common, resulting in a lot of noise in our input. If we can filter these out, then our network should have an easier time learning.

We also want to remove really rare words that show up in a only a few twits. Here you'll want to divide the count of each word by the **number of messages** calculated in the code block above (i.e. `len(messages))`. Then remove words that only appear in some small fraction of the messages.

>Note: There is not an exact number for low and high-frequency cut-offs, however there is a correct optimal range.
You should ideally set up low-frequency cut-off from 0.0000002 to 0.000007 (inclusive) and high-frequency from 5 to 20 (inclusive). If the number is too big, we lose lots of important words that we can use in our data.

In [11]:
"""
Set the following variables:
    freqs
    low_cutoff
    high_cutoff
    K_most_common
"""

# TODO Implement 

# Dictionart that contains the Frequency of words appearing in messages.
# The key is the token and the value is the frequency of that word in the corpus.
freqs = {k:v/len(messages) for k,v in bow.items()}

# print(freqs['raisinf'])

# Float that is the frequency cutoff. Drop words with a frequency that is lower or equal to this number.
low_cutoff = 0.0000002

# Integer that is the cut off for most common words. Drop words that are the `high_cutoff` most common words.
high_cutoff = 5

# The k most common words in the corpus. Use `high_cutoff` as the k.
K_most_common = [t[0] for t in bow.most_common(high_cutoff)]


filtered_words = [word for word in freqs if (freqs[word] > low_cutoff and word not in K_most_common)]
print(K_most_common)
len(filtered_words) 

['the', 'to', 'is', 'for', 'on']


98506

### Updating Vocabulary by Removing Filtered Words
Let's creat three variables that will help with our vocabulary.

In [12]:
"""
Set the following variables:
    vocab
    id2vocab
    filtered
"""

#TODO Implement

# A dictionary for the `filtered_words`. The key is the word and value is an id that represents the word. 
vocab = {v:i for i,v in enumerate(filtered_words,1)}
# Reverse of the `vocab` dictionary. The key is word id and value is the word. 
id2vocab = {v:i for i,v in vocab.items()}
# tokenized with the words not in `filtered_words` removed.
filtered = [[w for w in sentences if w not in K_most_common] for sentences in tokenized]

assert set(vocab.keys()) == set(id2vocab.values()), 'Check vocab and id2vocab dictionaries'

### Balancing the classes
Let's do a few last pre-processing steps. If we look at how our twits are labeled, we'll find that 50% of them are neutral. This means that our network will be 50% accurate just by guessing 0 every single time. To help our network learn appropriately, we'll want to balance our classes.
That is, make sure each of our different sentiment scores show up roughly as frequently in the data.

What we can do here is go through each of our examples and randomly drop twits with neutral sentiment. What should be the probability we drop these twits if we want to get around 20% neutral twits starting at 50% neutral? We should also take this opportunity to remove messages with length 0.

In [13]:
balanced = {'messages': [], 'sentiments':[]}

n_neutral = sum(1 for each in sentiments if each == 2)
N_examples = len(sentiments)
keep_prob = (N_examples - n_neutral)/4/n_neutral

for idx, sentiment in enumerate(sentiments):
    message = filtered[idx]
    if len(message) == 0:
        # skip this message because it has length zero
        continue
    elif sentiment != 2 or random.random() < keep_prob:
        balanced['messages'].append(message)
        balanced['sentiments'].append(sentiment) 

If you did it correctly, you should see the following result 

In [14]:
n_neutral = sum(1 for each in balanced['sentiments'] if each == 2)
N_examples = len(balanced['sentiments'])
n_neutral/N_examples

0.1953244137813287

Finally let's convert our tokens into integer ids which we can pass to the network.

In [15]:
# 'raisinf' in vocab.keys() by setting higher low_frq_thres we unable to retrieve some words like 'raisinf'

In [16]:
token_ids = [[vocab[word] for word in message] for message in balanced['messages']]
sentiments = balanced['sentiments']

## Neural Network
Now we have our vocabulary which means we can transform our tokens into ids, which are then passed to our network. So, let's define the network now!

Here is a nice diagram showing the network we'd like to build: 

#### Embed -> RNN -> Dense -> Softmax
### Implement the text classifier
Before we build text classifier, if you remember from the other network that you built in  "Sentiment Analysis with an RNN"  exercise  - which there, the network called " SentimentRNN", here we named it "TextClassifer" - consists of three main parts: 1) init function `__init__` 2) forward pass `forward`  3) hidden state `init_hidden`. 

This network is pretty similar to the network you built expect in the  `forward` pass, we use softmax instead of sigmoid. The reason we are not using sigmoid is that the output of NN is not a binary. In our network, sentiment scores have 5 possible outcomes. We are looking for an outcome with the highest probability thus softmax is a better choice.

In [17]:
class TextClassifier(nn.Module):
    def __init__(self, vocab_size, embed_size, lstm_size, output_size, lstm_layers=1, dropout=0.1):
        """
        Initialize the model by setting up the layers.
        
        Parameters
        ----------
            vocab_size : The vocabulary size.
            embed_size : The embedding layer size.
            lstm_size : The LSTM layer size.
            output_size : The output size.
            lstm_layers : The number of LSTM layers.
            dropout : The dropout probability.
        """
        
        super().__init__()
        self.vocab_size = vocab_size
        self.embed_size = embed_size
        self.lstm_size = lstm_size
        self.output_size = output_size
        self.lstm_layers = lstm_layers
        self.dropout = dropout
        
        # TODO Implement

        # Setup embedding layer
        self.embedding = nn.Embedding(vocab_size,embed_size)
        
        # Setup additional layers
        
        #using batch_first=True will make nn consider input size(0) as batch size
        #https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html?highlight=lstm#torch.nn.LSTM
        self.lstm = nn.LSTM(embed_size,lstm_size,lstm_layers,dropout=dropout,batch_first=False) 
        self.fc = nn.Linear(lstm_size,output_size)
        self.drop = nn.Dropout(dropout) # we need to add another drop out layer, if we have dropout param in LSTM?
        self.soft = nn.LogSoftmax(dim=1) #wrap softmax with log, using dim=1 for all columns prob added up to 1!!!


    def init_hidden(self, batch_size):
        """ 
        Initializes hidden state
        
        Parameters
        ----------
            batch_size : The size of batches.
        
        Returns
        -------
            hidden_state
            
        """
        
        # TODO Implement 
        
        # Create two new tensors with sizes n_layers x batch_size x hidden_dim,
        # initialized to zero, for hidden state and cell state of LSTM
        
        weight = next(self.parameters()).data
        
        if torch.cuda.is_available():
            hidden = (weight.new(self.lstm_layers,batch_size,self.lstm_size).zero_().cuda(),
                     weight.new(self.lstm_layers,batch_size,self.lstm_size).zero_().cuda())
        else:
             hidden = (weight.new(self.lstm_layers,batch_size,self.lstm_size).zero_(),
                     weight.new(self.lstm_layers,batch_size,self.lstm_size).zero_())
        
        return hidden


    def forward(self, nn_input, hidden_state):
        """
        Perform a forward pass of our model on nn_input.
        
        Parameters
        ----------
            nn_input : The batch of input to the NN.
            hidden_state : The LSTM hidden state.

        Returns
        -------
            logps: log softmax output
            hidden_state: The new hidden state.

        """
        
        # TODO Implement 
#         batch_size = nn_input.size(0)
#         seq_length = nn_input.size(1)
        
#         x = nn_input.view((batch_size,seq_length)).long()

        #x.size: (seq_length,batch_size)
        x = nn_input.long()
#         print("x size",x.size())
        #embedding
        x = self.embedding(x)
        lstm_out,hidden_state = self.lstm(x,hidden_state)
#         print("lstm old size",lstm_out.size())
        lstm_out = lstm_out[-1,:, :] # getting the last time step output
#         print("lstm size",lstm_out.size())
        out = self.fc(lstm_out)
#         print("out size",out.size())
        # softmax
        soft_out = self.soft(out)
#         print("softmax size",soft_out.size())
        
        return soft_out,hidden_state

In [18]:
m = nn.LogSoftmax()
m1 = nn.LogSoftmax(dim=0)
m2 = nn.LogSoftmax(dim=1)
input = torch.randn(2, 3)
output = m(input)
output1 = m1(input)
output2 = m2(input)
print("input",input)
print("output",output)
print("output1",output1)
print("o2",output2)
print("o exp",output.exp())
print("o1 exp",output1.exp())
print("o2 exp",output2.exp())

input tensor([[ 0.7051,  0.0455,  0.9724],
        [ 0.5740,  0.7019, -0.5033]])
output tensor([[-1.0379, -1.6976, -0.7707],
        [-0.9070, -0.7791, -1.9843]])
output1 tensor([[-0.6297, -1.0742, -0.2059],
        [-0.7609, -0.4179, -1.6816]])
o2 tensor([[-1.0379, -1.6976, -0.7707],
        [-0.9070, -0.7791, -1.9843]])
o exp tensor([[ 0.3542,  0.1831,  0.4627],
        [ 0.4037,  0.4588,  0.1375]])
o1 exp tensor([[ 0.5327,  0.3416,  0.8139],
        [ 0.4673,  0.6584,  0.1861]])
o2 exp tensor([[ 0.3542,  0.1831,  0.4627],
        [ 0.4037,  0.4588,  0.1375]])


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  """


### View Model

In [19]:
model = TextClassifier(len(vocab), 10, 6, 5, dropout=0.1, lstm_layers=2)
model.embedding.weight.data.uniform_(-1, 1)
model.to("cuda")
input = torch.randint(0, 1000, (5, 4), dtype=torch.int64).to("cuda")
hidden = model.init_hidden(4)

logps, _ = model.forward(input, hidden)
print(logps)

tensor([[-1.8627, -1.3836, -1.2610, -1.8382, -1.8864],
        [-1.8165, -1.3990, -1.2820, -1.8027, -1.9091],
        [-1.8366, -1.3978, -1.2703, -1.8213, -1.8908],
        [-1.8256, -1.4107, -1.2760, -1.8169, -1.8760]], device='cuda:0')


## Training
### DataLoaders and Batching
Now we should build a generator that we can use to loop through our data. It'll be more efficient if we can pass our sequences in as batches. Our input tensors should look like `(sequence_length, batch_size)`. So if our sequences are 40 tokens long and we pass in 25 sequences, then we'd have an input size of `(40, 25)`.

If we set our sequence length to 40, what do we do with messages that are more or less than 40 tokens? For messages with fewer than 40 tokens, we will pad the empty spots with zeros. We should be sure to **left** pad so that the RNN starts from nothing before going through the data. If the message has 20 tokens, then the first 20 spots of our 40 long sequence will be 0. If a message has more than 40 tokens, we'll just keep the first 40 tokens.

In [20]:
def dataloader(messages, labels, sequence_length=30, batch_size=32, shuffle=False):
    """ 
    Build a dataloader.
    """
    if shuffle:
        indices = list(range(len(messages)))
        random.shuffle(indices)
        messages = [messages[idx] for idx in indices]
        labels = [labels[idx] for idx in indices]

    total_sequences = len(messages)

    for ii in range(0, total_sequences, batch_size):
        batch_messages = messages[ii: ii+batch_size]
        
        # First initialize a tensor of all zeros
        batch = torch.zeros((sequence_length, len(batch_messages)), dtype=torch.int64)
        for batch_num, tokens in enumerate(batch_messages):
            token_tensor = torch.tensor(tokens)
            # Left pad!
            start_idx = max(sequence_length - len(token_tensor), 0)
            batch[start_idx:, batch_num] = token_tensor[:sequence_length]
        
        label_tensor = torch.tensor(labels[ii: ii+len(batch_messages)])
        
        yield batch, label_tensor

### Training and  Validation
With our data in nice shape, we'll split it into training and validation sets.

In [21]:
"""
Split data into training and validation datasets. Use an appropriate split size.
The features are the `token_ids` and the labels are the `sentiments`.
"""   

# TODO Implement 

l = (len(token_ids))//(20*64)

split_train = int(.8*l*(20*64))
split_valid = int(.2*l*(20*64))
print(split_train)

train_features = token_ids[:split_train]
valid_features = token_ids[split_train:split_train+split_valid]
train_labels = sentiments[:split_train]
valid_labels = sentiments[split_train:split_train+split_valid]

824320


In [22]:
# token_ids[:10]

In [23]:
text_batch, labels = next(iter(dataloader(train_features, train_labels, sequence_length=20, batch_size=64)))
text_batch, labels = text_batch.to("cuda"), labels.to("cuda")
print(text_batch.size())
#vocab_size, embed_size, lstm_size, output_size, lstm_layers=1, dropout=0.1
model = TextClassifier(len(vocab)+1, 200, 128, 5, dropout=0.)
model.to("cuda")
hidden = model.init_hidden(64)
logps, hidden = model.forward(text_batch, hidden)

torch.Size([20, 64])


In [24]:
# # dataloader(valid_features, valid_labels, batch_size=batch_size, sequence_length=20, shuffle=True)
# text_batch, labels = next(iter(dataloader(valid_features, valid_labels, sequence_length=20, batch_size=64)))
# text_batch, labels = text_batch.to("cuda"), labels.to("cuda")
# print(text_batch.size())
# model = TextClassifier(len(vocab)+1, 200, 128, 5, dropout=0.)
# model.to("cuda")
# hidden = model.init_hidden(64)
# logps, hidden = model.forward(text_batch, hidden)

### Training
It's time to train the neural network!

In [25]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = TextClassifier(len(vocab)+1, 1024, 512, 5, lstm_layers=2, dropout=0.2)
model.embedding.weight.data.uniform_(-1, 1)
model.to(device)

TextClassifier(
  (embedding): Embedding(98507, 1024)
  (lstm): LSTM(1024, 512, num_layers=2, dropout=0.2)
  (fc): Linear(in_features=512, out_features=5, bias=True)
  (drop): Dropout(p=0.2)
  (soft): LogSoftmax()
)

In [26]:
train_features[:10]

[[1, 2, 3, 4, 5],
 [6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 3, 16, 17, 18, 19, 2],
 [20, 21, 22, 23, 24, 25, 23, 26, 27, 24, 28, 29],
 [30, 31],
 [32, 33, 34, 35, 36, 37, 38, 39],
 [40, 41, 42, 43, 44, 45],
 [46, 47, 48, 49, 50, 51, 52, 15, 53, 54, 55],
 [56, 57, 48, 58, 59, 60, 61],
 [33, 62, 63, 64, 65, 66, 67, 68, 52, 69, 60, 66, 70, 71, 72, 73, 74],
 [75, 2]]

In [27]:
"""
Train your model with dropout. Make sure to clip your gradients.
Print the training loss, validation loss, and validation accuracy for every 100 steps.
"""

epochs = 1
batch_size = 64
learning_rate = 0.0001
clip=5

print_every = 100
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
model.train()

for epoch in range(epochs):
    print('Starting epoch {}'.format(epoch + 1))
    
    hidden = model.init_hidden(batch_size)
    steps = 0
    for text_batch, labels in dataloader(
            train_features, train_labels, batch_size=batch_size, sequence_length=20, shuffle=True):
        steps += 1
        
        # Set Device
        text_batch, labels = text_batch.to(device), labels.to(device)
        for each in hidden:
            each.to(device)
        
        # TODO Implement: Train Model
        hidden = tuple([each.data for each in hidden])
        
        #zero gradient
        model.zero_grad()
        
        
        #forward
        output,hidden = model(text_batch,hidden)
        
        #loss
        loss = criterion(output.squeeze(),labels.long())
        
        #backward, calc gradient
        loss.backward()
        
        #clip gradient
        nn.utils.clip_grad_norm_(model.parameters(),clip)
        
     
        #update weight
        optimizer.step()
        
        if steps % print_every == 0:
            # eval model
            model.eval()
            
            #check fc gradient test
            weight_grad = model.fc.weight.grad
            bias_grad = model.fc.bias.grad
            print(f"weight gradient of fc is {weight_grad}, bais gradient of fc is {bias_grad}")

            
            # TODO Implement: Print metrics
            val_h = model.init_hidden(batch_size)
            val_losses = []
            
            for val_text_batch, val_labels in dataloader(
                valid_features, valid_labels, batch_size=batch_size, sequence_length=20, shuffle=True):
                
                
                # detach hidden from history
                val_h = tuple([each.data for each in val_h])
                
#                 print(val_text_batch.size())
                
                # Set Device
                val_text_batch, val_labels = val_text_batch.to(device), val_labels.to(device)
                for each in val_h:
                    each.to(device)
                    
                val_out,val_h = model(val_text_batch,val_h)
                val_loss = criterion(val_out.squeeze(),val_labels.long())
                
                val_losses.append(val_loss.item())
                
            model.train()
            
            print(f"Epoch: {epoch+1}/{epochs}...",
                 f"Step: {steps}...",
                 "Loss: {:.6f}...".format(loss.item()),
                 "Val Loss:{:.6f}".format(np.mean(val_losses)))

Starting epoch 1
weight gradient of fc is tensor([[ 4.0213e-03,  8.4290e-04,  9.9284e-04,  ...,  4.9341e-03,
         -9.3166e-03, -3.7850e-04],
        [-2.3949e-03, -2.0587e-03,  5.6994e-04,  ..., -6.2658e-03,
          3.1732e-03,  3.1332e-03],
        [-2.5663e-03, -3.1609e-03,  1.9551e-04,  ..., -1.8645e-03,
          2.5507e-03,  2.2665e-03],
        [-1.8275e-03, -1.4696e-03,  1.7441e-03,  ..., -5.8661e-04,
          2.8173e-03,  2.5979e-03],
        [ 2.7674e-03,  5.8463e-03, -3.5024e-03,  ...,  3.7828e-03,
          7.7538e-04, -7.6191e-03]], device='cuda:0'), bais gradient of fc is tensor(1.00000e-02 *
       [-9.0625,  2.0123,  1.6804,  3.3173,  2.0525], device='cuda:0')
Epoch: 1/1... Step: 100... Loss: 1.458937... Val Loss:1.491462
weight gradient of fc is tensor(1.00000e-02 *
       [[ 0.1168,  0.6288, -0.8850,  ...,  0.4482,  0.4092, -0.5123],
        [-0.0177, -0.4566,  1.5326,  ..., -0.6450, -0.5339,  0.5950],
        [ 0.3715, -0.1266,  0.1392,  ...,  0.1101, -0.0511, 

## Making Predictions
### Prediction 
Okay, now that you have a trained model, try it on some new twits and see if it works appropriately. Remember that for any new text, you'll need to preprocess it first before passing it to the network. Implement the `predict` function to generate the prediction vector from a message.

In [28]:
def predict(text, model, vocab):
    """ 
    Make a prediction on a single sentence.

    Parameters
    ----------
        text : The string to make a prediction on.
        model : The model to use for making the prediction.
        vocab : Dictionary for word to word ids. The key is the word and the value is the word id.

    Returns
    -------
        pred : Prediction vector
    """    
    
    # TODO Implement
    
    tokens = preprocess(text)
    
    # Filter non-vocab words
    tokens = [t for t in tokens if t in vocab.keys()]
    # Convert words to ids
    tokens = [vocab[t] for t in tokens]
        
    # Adding a batch dimension
    text_input = torch.from_numpy(np.asarray(tokens)).view(-1,1)
    
    text_input = text_input.to("cuda")
    # Get the NN output
    hidden = model.init_hidden(1)
    logps, _ = model.forward(text_input,hidden)
    # Take the exponent of the NN output to get a range of 0 to 1 for each label.
    pred = logps.exp()
    
    return pred

In [29]:
text = "Google is working on self driving cars, I'm bullish on $goog"
model.eval()
model.to("cuda")
predict(text, model, vocab)

tensor([[ 0.0062,  0.0857,  0.0269,  0.4101,  0.4710]], device='cuda:0')

### Questions: What is the prediction of the model? What is the uncertainty of the prediction?
** TODO: Answer Question**

Now we have a trained model and we can make predictions. We can use this model to track the sentiments of various stocks by predicting the sentiments of twits as they are coming in. Now we have a stream of twits. For each of those twits, pull out the stocks mentioned in them and keep track of the sentiments. Remember that in the twits, ticker symbols are encoded with a dollar sign as the first character, all caps, and 2-4 letters, like $AAPL. Ideally, you'd want to track the sentiments of the stocks in your universe and use this as a signal in your larger model(s).

## Testing
### Load the Data 

In [30]:
with open(os.path.join('..', '..', 'data', 'project_6_stocktwits', 'test_twits.json'), 'r') as f:
    test_data = json.load(f)

### Twit Stream

In [31]:
def twit_stream():
    for twit in test_data['data']:
        yield twit

next(twit_stream())

{'message_body': '$JWN has moved -1.69% on 10-31. Check out the movement and peers at  https://dividendbot.com?s=JWN',
 'timestamp': '2018-11-01T00:00:05Z'}

Using the `prediction` function, let's apply it to a stream of twits.

In [32]:
def score_twits(stream, model, vocab, universe):
    """ 
    Given a stream of twits and a universe of tickers, return sentiment scores for tickers in the universe.
    """
    for twit in stream:

        # Get the message text
        text = twit['message_body']
        symbols = re.findall('\$[A-Z]{2,4}', text)
        score = predict(text, model, vocab)

        for symbol in symbols:
            if symbol in universe:
                yield {'symbol': symbol, 'score': score, 'timestamp': twit['timestamp']}

In [33]:
universe = {'$BBRY', '$AAPL', '$AMZN', '$BABA', '$YHOO', '$LQMT', '$FB', '$GOOG', '$BBBY', '$JNUG', '$SBUX', '$MU'}
score_stream = score_twits(twit_stream(), model, vocab, universe)

next(score_stream)

{'symbol': '$AAPL',
 'score': tensor([[ 0.1123,  0.1684,  0.1974,  0.2688,  0.2531]], device='cuda:0'),
 'timestamp': '2018-11-01T00:00:18Z'}

That's it. You have successfully built a model for sentiment analysis! 

## Submission
Now that you're done with the project, it's time to submit it. Click the submit button in the bottom right. One of our reviewers will give you feedback on your project with a pass or not passed grade. You can continue to the next section while you wait for feedback.